In [ ]:
#Imports das classes criadas
from Classes.Arquivo import Arquivo
from Classes.BancoDados import BancoDados
from Classes.BancoDDL import BancoDDL
from Classes.BancoDML import BancoDML
from Classes.BancoDQL import BancoDQL
from Classes.AnaliseGenero import AnaliseGenero
from Classes.AnaliseArmaFogo import AnaliseArmaFogo
from Classes.AnaliseRegiao import AnaliseRegiao
import plotly.express as px #py -3.12 -m pip install plotly
import pandas as pd

In [2]:
#Carregando todos os CSVs e os inspecionando
try:
    #Criação dos dataframes
    s_total = Arquivo("Arquivos CSV/suicidios-total.csv")
    homens_arma = Arquivo("Arquivos CSV/suicidios-homens-armas-fogo.csv")
    mulheres_arma = Arquivo("Arquivos CSV/suicidios-mulheres-armas-fogo.csv")
    s_homens = Arquivo("Arquivos CSV/suicidios-homens.csv")
    s_mulheres = Arquivo("Arquivos CSV/suicidios-mulheres.csv")
    s_regiao = Arquivo("Arquivos CSV/suicidios-região.csv")

    #Inspeção
    if s_total.verifica and homens_arma.verifica and mulheres_arma.verifica and s_homens.verifica and s_mulheres.verifica and s_regiao.verifica:
        print("Arquivos CSV inspecionados com sucesso!")
    else:
        print(f"""
Status de verificação de cada arquivo:
suicidios-total.csv = {s_total.verifica}
suicidios-homens-armas-fogo.csv = {homens_arma.verifica}
suicidios-mulheres-armas-fogo.csv = {mulheres_arma.verifica}
suicidios-homens.csv = {s_homens.verifica}
suicidios-mulheres.csv = {s_mulheres.verifica}
suicidios-região.csv = {s_regiao.verifica}
        """)
except Exception as e:
    print(f"Erro ao inspecionar o arquivo: {e}")

Arquivos CSV inspecionados com sucesso!


In [3]:
#Abrindo conexão com banco local
BancoLocal = BancoDQL('PPCD_LAW', 'postgres', '123456', 'localhost', '5432')

Conexão bem-sucedida!


In [ ]:
#Criando tabelas no banco local (localhost) PostgresSQL
#CriacaoBanco = BancoDDL('PPCD_LAW', 'postgres', '123456', 'localhost', '5432')
#CriacaoBanco.criacao_tabelas()

In [4]:
#Instanciando DataFrames para usar na inserção dos dados
colunasPeriodo = BancoLocal.buscar_dados('periodo')[['per_cod', 'ano']]
colunasRegiao = BancoLocal.buscar_dados('regiao')[['reg_cod', 'nome']]
regiaoDf = s_regiao.dataframe[['nome', 'período', 'valor']]
mulherFogoDf = mulheres_arma.dataframe[['período', 'valor']]
homemFogoDf = homens_arma.dataframe[['período', 'valor']]

In [ ]:
#Realizando inserção dos dados no banco de dados local
BancoLocalDML = BancoDML('PPCD_LAW', 'postgres', '123456', 'localhost', '5432')
#Table periodo:
#BancoLocalDML.inserir_total(s_total.dataframe)

#Table gen_periodo:
#BancoLocalDML.inserir_gen(s_homens.dataframe, s_mulheres.dataframe, colunasPeriodo)

#Table reg_periodo:
#BancoLocalDML.inserir_reg(colunasRegiao, regiaoDf,  colunasPeriodo)

#Table arma_fogo:
#BancoLocalDML.inserir_dados_fogo(mulherFogoDf, homemFogoDf, colunasPeriodo)
BancoLocalDML.encerrar()

In [ ]:
#Gráfico dos gêneros
dadosGeneros = BancoLocal.buscar_dados('gen_periodo')
dadosSuicidiosGeral = s_total.dataframe['valor'].sum() - dadosGeneros['quantidade'].sum()

soma_generos = dadosGeneros.groupby(['gen_cod','descricao','tipo'], as_index=False).sum()
novo_dado = pd.DataFrame({'gen_cod': [3], 'quantidade': [dadosSuicidiosGeral], 'descricao': ['IGN']})
dados_final = pd.concat([soma_generos, novo_dado], ignore_index=True)

analiseGenero = AnaliseGenero(dados_final, 'quantidade', 'descricao', 'quantidade', 'pie')
analiseGenero.criar_grafico()

In [7]:
#vertical bar label por genero por periodo
dadosGeneros = BancoLocal.buscar_dados('gen_periodo')

dadosGeneros = dadosGeneros.rename(columns={'descricao':'Gêneros', 'quantidade':'Quantidade', 'ano':'Período'})

analiseGeneroPeriodo = AnaliseGenero(dadosGeneros, 'Período', 'Quantidade', 'Gêneros', 'histogram')
analiseGeneroPeriodo.criar_grafico()

In [ ]:
analiseGeneroPoli = AnaliseGenero(dadosGeneros, 'quantidade', 'descricao', 'quantidade', 'pie')
analiseGeneroPoli.modelo_regre_poli(grau_homens=2, grau_mulheres=2)

Modelo Polinomial - Homens
Erro Quadrático Médio (MSE): 154423.11684669892
Previsão de suicídios (homens) 2023-2030: [12161.56517408 12562.80797394 12973.26593487 13392.93905686
 13821.82733992 14259.93078404 14707.24938922 15163.78315547]
Desvio padrão - Homens: 392.96706839975656

Modelo Polinomial - Mulheres
Erro Quadrático Médio (MSE): 12460.514851464446
Previsão de suicídios (mulheres) 2023-2030: [3429.02506684 3562.00950152 3698.88747771 3839.65899541 3984.32405461
 4132.88265533 4285.33479755 4441.68048127]
Desvio padrão - Mulheres: 111.62667625377209


In [9]:
#quantidade por regiao em cada periodo (linegraph) 
dadosRegPeriodo = BancoLocal.buscar_dados('reg_periodo')

dadosRegPeriodo = dadosRegPeriodo.rename(columns={'quantidade':'Quantidade','ano':'Período','descricao':'Descrição'})

analiseRegiaoPeriodo = AnaliseRegiao(dadosRegPeriodo, 'Período', 'Quantidade', 'Descrição')
analiseRegiaoPeriodo.criar_grafico()
analiseRegiaoPeriodo.anos_max_min()

Resultados (Maior e Menor Valor por Região):
      Descrição  Ano_Maior  Maior_Valor  Ano_Menor  Menor_Valor
0  Centro-Oeste       2022         4930       2017         1781
1       Sudeste       2013         4898       2017         1349
2           Sul       2021         4690       2016         1311
3      Nordeste       2020         4391       2017         1035
4         Norte       2013         4764       2021         1132


In [10]:
dadosArmaFogo = BancoLocal.buscar_dados('arma_fogo')
df = px.data.gapminder()
dadosArmaFogo["size_scaled"] = (dadosArmaFogo["quantidade_total"] - dadosArmaFogo["quantidade_total"].min()) / \
                               (dadosArmaFogo["quantidade_total"].max() - dadosArmaFogo["quantidade_total"].min()) * 50
dadosArmaFogo = dadosArmaFogo.rename(columns={'ano':'Período', 'quantidade_total':'Quantidade'})
analiseArma = AnaliseArmaFogo(dadosArmaFogo, 'Período', 'Quantidade', 'Quantidade')
analiseArma.criar_grafico()